In [ ]:
import rospy
from giskardpy.python_interface.python_interface import GiskardWrapper
import numpy as np
from geometry_msgs.msg import PoseStamped, Point, Quaternion, Vector3Stamped
from tf.transformations import quaternion_from_matrix, quaternion_about_axis, quaternion_multiply
from giskardpy.goals.adaptive_goals import CloseGripper, PouringAdaptiveTilt
import math
from giskardpy.goals.manipulability_goals import MaxManipulability

In [ ]:
rospy.init_node('giskard_notebook')

In [ ]:
giskard = GiskardWrapper()

In [ ]:
p = PoseStamped()
p.header.stamp = rospy.get_rostime()
p.header.frame_id = 'map'

rotation = 0  # .9
rot1 = quaternion_about_axis(rotation, (0, 1, 0))
rot2 = quaternion_about_axis(rotation, (0, 1, 0))

h = 0.9
p.pose.position = Point(2, -0.39, h)
p.pose.orientation = Quaternion(*quaternion_multiply(rot1, quaternion_from_matrix([[0, 0, 1, 0],
                                                                                   [1, 0, 0, 0],
                                                                                   [0, 1, 0, 0],
                                                                                   [0, 0, 0, 1]])))

p2 = PoseStamped()
p2.header.stamp = rospy.get_rostime()
p2.header.frame_id = 'map'
p2.pose.position = Point(2, -0.0, h)
p2.pose.orientation = Quaternion(*quaternion_multiply(rot2, quaternion_from_matrix([[0, 0, -1, 0],
                                                                                    [-1, 0, 0, 0],
                                                                                    [0, 1, 0, 0],
                                                                                    [0, 0, 0, 1]])))

####################################################
h = 0.01
p.header.frame_id = 'pot'
p2.header.frame_id = 'pot'
p.pose.position = Point(0, -0.20, h)
p2.pose.position = Point(0, 0.20, h)
giskard.motion_goals.allow_all_collisions()
giskard.motion_goals.add_cartesian_pose(p, 'r_gripper_tool_frame', 'map')
giskard.motion_goals.add_cartesian_pose(p2, 'l_gripper_tool_frame', 'map')
giskard.add_default_end_motion_conditions()
giskard.execute()

###################################################

p.pose.position = Point(0, -0.15, h)
p2.pose.position = Point(0, 0.15, h)
giskard.motion_goals.allow_all_collisions()
giskard.motion_goals.add_cartesian_pose(p, 'r_gripper_tool_frame', 'map')
giskard.motion_goals.add_cartesian_pose(p2, 'l_gripper_tool_frame', 'map')
giskard.add_default_end_motion_conditions()
giskard.execute()

###################################################

giskard.motion_goals.add_motion_goal(motion_goal_class=CloseGripper.__name__,
                                       name='closeGripperRight',
                                       pub_topic='/pr2/r_gripper_controller/command',
                                       joint_state_topic='pr2/joint_states',
                                       alibi_joint_name='r_gripper_l_finger_joint',
                                       effort_threshold=-0.14,
                                       effort=-180)
giskard.motion_goals.add_motion_goal(motion_goal_class=CloseGripper.__name__,
                                       name='closeGripperLeft',
                                       pub_topic='/pr2/l_gripper_controller/command',
                                       joint_state_topic='pr2/joint_states',
                                       alibi_joint_name='l_gripper_l_finger_joint',
                                       effort_threshold=-0.14,
                                       effort=-180)
giskard.motion_goals.add_cartesian_pose(p, 'r_gripper_tool_frame', 'map')
giskard.motion_goals.add_cartesian_pose(p2, 'l_gripper_tool_frame', 'map')
giskard.add_default_end_motion_conditions()
giskard.motion_goals.allow_all_collisions()
giskard.execute()

####################################################

# current pose of the pot in simulation
pot_pose = PoseStamped()
pot_pose.header.frame_id = 'pot'
pot_pose.pose.position = Point(0, 0, 0)
pot_pose.pose.orientation.w = 1

# add a new object at the pose of the pot and attach it to the right tip
giskard.world.add_box('dummy', (0.1, 0.1, 0.1), pose=pot_pose, parent_link='r_gripper_tool_frame')

# pose of the left gripper relative to the attached object
l_pose = PoseStamped()
l_pose.header.frame_id = 'dummy'
l_pose.pose.position = Point(0, 0.15, 0)
l_pose.pose.orientation = Quaternion(*quaternion_from_matrix([[0, 0, -1, 0],
                                                              [-1, 0, 0, 0],
                                                              [0, 1, 0, 0],
                                                              [0, 0, 0, 1]]))

# update pot pose to the desired pose of the pot
# pot_pose.pose.position.z = 0.2
#
# move the dummy pot and the left gripper relative to it
pot_pose.pose.position = Point(0, 0, 0.2)
giskard.motion_goals.add_cartesian_pose(pot_pose, 'dummy', 'map')
giskard.motion_goals.add_cartesian_pose(l_pose, 'l_gripper_tool_frame', 'r_gripper_tool_frame', add_monitor=False)
giskard.add_default_end_motion_conditions()
giskard.execute()

###############################################################

pot_pose = PoseStamped()
pot_pose.header.frame_id = 'map'
pot_pose.pose.position = Point(1.9, 0.3, 1.1)
pot_pose.pose.orientation.w = 1  # Quaternion(*quaternion_about_axis(-0.5, (1, 0, 0)))
# Todo: rotate pot pose around x axis
tilt_axis = Vector3Stamped()
tilt_axis.header.frame_id = 'dummy'
tilt_axis.vector.y = 2 / math.sqrt(5)
tilt_axis.vector.x = -1 / math.sqrt(5)
# tilt_axis.vector.y = 1
giskard.motion_goals.add_motion_goal(motion_goal_class=PouringAdaptiveTilt.__name__,
                                       name='pouring',
                                       tip='dummy',
                                       root='map',
                                       tilt_angle=0.7,
                                       pouring_pose=pot_pose,
                                       tilt_axis=tilt_axis,
                                       pre_tilt=False)
giskard.motion_goals.add_cartesian_pose(l_pose, 'l_gripper_tool_frame', 'r_gripper_tool_frame', add_monitor=False)
giskard.motion_goals.allow_all_collisions()
# giskard.set_avoid_joint_limits_goal()
giskard.motion_goals.add_motion_goal(motion_goal_class=MaxManipulability.__name__,
                                       root_link='torso_lift_link',
                                       tip_link='r_gripper_tool_frame',
                                       gain=3)
giskard.motion_goals.add_motion_goal(motion_goal_class=MaxManipulability.__name__,
                                       root_link='torso_lift_link',
                                       tip_link='l_gripper_tool_frame',
                                       gain=3)
giskard.execute(add_local_minimum_reached=False)